In [1]:
%matplotlib notebook

# Project Euler problem 688 - Piles of Plates

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=688)

## Description

We stack $n$ plates into $k$ non-empty piles where each pile is a different size. Define $f(n,k)$ to be the maximum number of plates possible in the smallest pile. For example when $n=10$ and $k=3$ the piles $2,3,5$ is the best that can be done and so $f(10,3)=2$. It is impossible to divide 10 into 5 non-empty piles and hence $f(10,5)=0$.

Define $F(n)$ to be the sum of $f(n,k)$ for all possible pile sizes $k\geq 1$. For example $F(100)=275$.

Further define $S(N)= \sum_{n=1}^N F(n)$. You are given $S(100)=12656$.

Find $S(10^{16})$ giving your answer modulo $1\,000\,000\,007$.

## Initial investigation

for $k$ piles the smallest number of plates in all piles so that all piles are different and no piles are empty is given by 

$$1 + 2 + \ldots + k = \frac{k(k+1)}{2}.$$

In this case the number of plates in the smallest pile is 1.

In general, $f$ can be calculated as

$$f(n, k) = \max(0, \frac{n-\frac{(k-1)k}{2}}{k}) $$

with the maximum number of piles, $k$, for a given number of plates given as

$$k_\mathrm{max} = \left\lfloor\frac{\sqrt{8n + 1} - 1}{2} \right\rfloor. $$

With this, we can calculate brute-force versions of all the functions.

In [2]:
from math import sqrt

def kmax(n):
    return int((sqrt(8*n + 1) - 1)/2)

def f(n, k):
    return max(0, (n-(k-1)*k//2)//k)

def F(n):
    total = 0
    for k in range(1, kmax(n)+1):
        total += f(n, k)
    return total

def S(N):
    total = 0
    for n in range(1, N+1):
        total += F(n)
    return total

print("f(10, 3) = {}".format(f(10, 3)))
print("f(10, 5) = {}".format(f(10, 5)))
print("F(100) = {}".format(F(100)))
print("S(100) = {}".format(S(100)))

f(10, 3) = 2
f(10, 5) = 0
F(100) = 275
S(100) = 12656


While the algorithm works, this will be too slow to iterate to $n = 10^{16}$. However, we note that the maximum number of piles for $n = 10^{16}$ is $k_\mathrm{max} = 141\,421\,355$, which is small enough to iterate over. Thus, if we can find a closed form solution for the sum of plates for constant $k$ the problem becomes tractable.

Define a function $m(n_\max, k) = \sum_{n=n_\min}^{n_\max} f(n, k)$, where $n_\min = \frac{k(k+1)}{2}$ is the minimum possible total number of plates for given k. This function can be calculated without the need to iterate over $n$.

In [3]:
def nmin(k):
    return k*(k+1)//2

def m(nmax, k):
    _nmin = nmin(k)
    if _nmin >= nmax:
        return 0
    
    ndiff, rem = divmod(nmax - _nmin + 1, k)
    
    return k*nmin(ndiff) + (ndiff+1)*rem

The final step, to iterate over all possible valures of $k$ can be done by brute force

In [6]:
def M(n, mod=10**9+7):
    total = 0
    for k in range(1, kmax(n)+1):
        total += m(n, k) % mod
    return total % mod

print("M(10**6) % 1000000007 = {}".format(M(10**16)))
%timeit M(10**16)

M(10**6) % 1000000007 = 110941813
2min 7s ± 1.47 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In pure Python this takes a few minutes to run, while it runs in roughly 30 seconds in pypy.